In [2]:
import numpy as np

input_file = "big.in"
data = open(input_file, "r").read()

data = data.split()
R = int(data[0])
C = int(data[1])
L = int(data[2])  # at least L cells of each ingredient
H = int(data[3])  # at most H cells in total
pizza = np.array([[0 if ingredient == "T" else 1 for ingredient in row] for row in data[4:]], dtype=np.int8)

In [3]:
R, C , L, H

(1000, 1000, 6, 14)

In [37]:
from pulp import LpProblem, LpVariable, LpMaximize, LpAffineExpression
from pulp.solvers import GLPK
from sympy import divisors

def score(solution):
    return sum([(s[2] - s[0] + 1) * (s[3] - s[1] + 1) for s in solution])

def solve(pizza, L, H, offset_rows=0, offset_cols=0):  
    # enumerate all possible sizes
    # min == 2*L  max == H
    sizes = []

    for n_cells in range(2 * L, H + 1):
        for div in divisors(n_cells):
            sizes.append((div, n_cells // div))
            
    # enumerate all subsets
    subsets = []
    for rows, cols in sizes:
        for i in range(pizza.shape[0] - rows + 1):
            for j in range(pizza.shape[1] - cols + 1):                    
                toppings = pizza[i:i+rows, j:j+cols].sum()  
                if toppings >= L and rows*cols - toppings >= L:
                    subsets.append((i, j, i+rows, j+cols))
                    
    if len(subsets) == 0:
        return []

    subsets_idx = ["%d_%d_%d_%d" % (a, b, c, d) for a, b, c, d in subsets]
    
    # constraints
    constraints = {}

    for i, s in enumerate(subsets):
        for c_i in range(s[0], s[2]):
            for c_j in range(s[1], s[3]):
                if (c_i, c_j) not in constraints:
                    constraints[(c_i, c_j)] = []
                constraints[(c_i, c_j)].append(i)    

    # chop off singletons
    constraints = {k:v for k, v in constraints.items() if len(v) > 1}
    
    # linear program
    problem = LpProblem(name="pizza", sense=LpMaximize)
    
    # variables
    slices = LpVariable.dicts("slices", subsets_idx, lowBound=0, upBound=1, cat="Binary")

    # objective
    exp = LpAffineExpression()
    for i, s in enumerate(subsets):
        exp += (s[2]-s[0]) * (s[3]-s[1]) * slices[subsets_idx[i]]
    problem += exp

    # constraints
    for pos, ids in constraints.items():
        exp = LpAffineExpression()
        for i in ids:
            exp += slices[subsets_idx[i]]
        problem += exp <= 1

    # solve
    try:
        problem.solve(GLPK())
    except:
        return []
    
    # extract solution
    solution = []

    for i, s in enumerate(subsets):
        if slices[subsets_idx[i]].value() > 0.0:
            solution.append((s[0]+offset_rows, s[1]+offset_cols, 
                             s[2]-1+offset_rows, s[3]-1+offset_cols))
    
    # debug info
    print(offset_rows, offset_cols, score(solution))
            
    return solution

In [22]:
# Greedy approximation by solving on 20x20 sub-pizzas and by recombining solutions
from joblib import Parallel, delayed

step = 20

solution = Parallel(n_jobs=4, verbose=3)(
    delayed(solve)(pizza[i:i+step, j:j+step], L, H, offset_rows=i, offset_cols=j)
                   for i in range(0, pizza.shape[0], step)
                   for j in range(0, pizza.shape[1], step))

solution = sum(solution, [])

0 0 366
0 40 395
0 80 381
0 60 386
0 20 388
0 160 400
0 120 398
0 180 362
0 100 378
0 200 396
0 220 384
0 260 387
0 140 391
0 320 394
0 280 387
0 300 392
0 240 397
0 360 382
0 380 380
0 420 398
0 460 393
0 440 364
0 480 381
0 400 365


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   48.1s


0 340 376
0 560 392
0 580 397
0 500 386
0 600 381
0 640 399
0 520 389
0 680 393
0 700 398
0 720 382
0 620 378
0 760 379
0 540 392
0 780 381
0 740 390
0 800 393
0 860 388
0 880 385
0 820 362
0 920 364
0 900 377
0 960 389
0 940 366
0 980 398
0 840 382
20 40 389
20 20 377
20 0 389
20 100 379
20 120 398
20 80 386
20 60 358
20 140 392
20 200 386
20 180 381
20 160 387
20 240 382
20 280 388
20 220 375
20 320 377
20 340 391
20 360 360
20 380 369
20 400 385
20 420 396
20 300 376
20 260 387
20 440 384
20 460 392
20 500 393
20 520 388
20 540 400
20 560 377
20 480 378
20 600 395
0 660 386
20 580 384
20 620 382
20 700 398
20 660 384
20 640 395
20 740 378
20 680 394
20 800 399
20 780 390
20 760 376
20 820 398
20 720 378
20 840 378
20 880 395
20 900 388
20 920 390
20 980 392
40 0 388
40 20 400
20 860 380
40 40 364
40 80 400
20 960 384
20 940 370
40 60 382
40 100 374
40 180 395
40 200 379
40 160 379
40 240 379
40 220 392
40 140 383
40 300 393
40 320 392
40 280 374
40 340 382
40 360 380
40 400 389
40 3

[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  5.1min


40 260 398
40 420 372
40 500 385
40 520 393
40 120 396
40 560 400
40 460 390
40 480 388
40 620 379
40 600 388
40 540 391
40 680 375
40 640 382
40 660 374
40 700 380
40 580 389
40 720 366
40 780 391
40 760 387
40 820 359
40 860 390
40 800 385
40 880 390
40 840 370
40 920 380
40 940 397
40 980 397
40 740 397
40 960 388
60 0 379
40 900 381
60 20 398
60 60 394
60 120 343
60 100 382
60 160 398
60 80 372
60 140 386
60 200 366
60 180 368
60 260 391
60 40 392
60 300 399
60 280 372
60 320 378
60 220 386
60 240 353
60 340 368
60 360 384
60 420 376
60 440 377
60 380 385
60 500 384
60 520 390
60 480 382
60 560 393
60 580 393
60 600 384
60 460 375
60 620 390
60 400 391
60 680 392
60 640 398
60 700 378
60 740 390
60 720 368
60 760 375
60 780 394
60 540 383
60 660 385
60 800 394
60 860 388
60 840 389
60 900 388
60 820 382
60 880 381
60 940 394
60 980 374
60 920 382
60 960 391
80 60 387
80 80 398
80 0 387
80 100 378
80 20 378
80 120 382
80 40 373
80 180 396
80 200 388
80 240 392
80 220 384
80 160 394


...

[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed: 73.2min


620 180 386
620 280 380
620 220 387
620 240 378
620 300 373
620 320 393
620 360 394
620 380 368
620 340 392
620 400 364
620 460 380
620 260 386
620 480 393
620 500 400
620 520 383
620 420 388
620 440 394
620 540 385
620 600 371
620 580 374
620 620 383
620 660 395
620 700 391
620 720 380
620 680 368
620 640 383
620 760 395
620 740 386
620 800 391
620 780 378
620 840 399
620 560 384
620 880 380
620 820 390
620 860 380
620 900 380
620 980 387
620 960 366
640 0 394
620 940 380
640 60 394
640 20 392
640 40 378
620 920 394
640 140 397
640 160 385
640 100 391
640 200 398
640 80 384
640 120 394
640 220 389
640 240 370
640 300 376
640 260 395
640 280 355
640 340 391
640 320 392
640 400 384
640 360 380
640 380 386
640 180 386
640 460 388
640 420 376
640 480 364
640 540 382
640 560 393
640 500 393
640 600 382
640 580 389
640 440 383
640 520 385
640 680 389
640 640 389
640 620 391
640 700 395
640 740 398
640 660 377
640 800 397
640 780 395
640 720 378
640 860 400
640 880 382
640 900 400
640 840 38

[Parallel(n_jobs=4)]: Done 2040 tasks      | elapsed: 95.5min


800 820 383
800 880 390
800 840 380
800 920 377
800 740 385
800 960 388
800 980 376
800 900 366
820 0 388
820 20 383
800 860 372
820 40 393
820 100 368
820 120 367
820 140 399
800 940 381
820 80 396
820 160 386
820 60 395
820 220 382
820 200 368
820 240 388
820 280 378
820 300 387
820 260 361
820 180 378
820 320 397
820 400 397
820 380 385
820 340 392
820 360 380
820 480 379
820 500 387
820 440 396
820 460 375
820 540 381
820 580 379
820 560 393
820 600 367
820 640 394
820 520 393
820 680 396
820 700 391
820 420 376
820 620 391
820 740 386
820 720 382
820 780 364
820 660 384
820 760 382
820 860 398
820 820 394
820 800 390
820 920 389
820 900 392
820 960 399
820 880 385
820 980 393
820 840 375
840 20 367
840 40 396
840 60 391
840 80 396
840 100 391
840 140 380
840 160 372
840 0 385
820 940 391
840 220 383
840 200 387
840 260 391
840 280 391
840 120 357
840 180 387
840 300 397
840 340 396
840 360 330
840 380 386
840 420 385
840 400 383
840 240 381
840 320 384
840 440 384
840 500 383
840 

[Parallel(n_jobs=4)]: Done 2500 out of 2500 | elapsed: 115.5min finished


In [17]:
# input_file = "big.in-solution"
# fd = open(input_file, "r")
# n_slices = int(fd.readline())
# solution = []

# for i in range(n_slices):
#     s = fd.readline().split()
#     s = list(map(int, s))
#     solution.append(s)

In [41]:
print(score(solution))

961998


In [26]:
fd = open("%s-solution" % input_file, "w")
fd.write("%d\n" % len(solution))
for s in solution:
    fd.write("%d %d %d %d\n" % (s[0], s[1], s[2], s[3]))
fd.close()